In [2]:
import pandas as pd
import os
import numpy as np
from tabulate import tabulate
import dlib
import cv2
from tqdm import tqdm
import zipfile
from PIL import Image
import matplotlib.pyplot as plt
from skimage import filters
from tqdm import tqdm
import plotly.express as px

ModuleNotFoundError: No module named 'dlib'

In [3]:
zip_path = r"C:\Users\LEGION\Downloads\Agedetection\age_detection.zip"  # Update this with the correct path

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()

# Read the CSV file
df = pd.read_csv('age_detection.csv')

# Display the first few rows of the dataframe
print(df.head())

# Get the file path of the first image
image_path = df.loc[0, 'file']

# Check if the image path does not contain the absolute path, if not prepend the path
if not os.path.isabs(image_path):
    image_path = os.path.join(os.path.dirname(zip_path), image_path)

# Open and display the image
try:
    img = Image.open(image_path)
    img.show()
except Exception as e:
    print(f"An error occurred: {e}")


NameError: name 'zipfile' is not defined

In [4]:
print(df.columns) 
print(df.shape)  

NameError: name 'df' is not defined

# Data Cleaning

In [5]:
# File Path Validation
df = df[df['file'].apply(lambda x: os.path.isfile(x))]

# Duplicate Removal
df.drop_duplicates(subset=['file'], keep='first', inplace=True)

NameError: name 'df' is not defined

In [6]:
# Split Consistency
df = df[df['split'].isin(['train', 'test'])]

NameError: name 'df' is not defined

In [7]:
# Image Quality
# removin images that are too small
min_width, min_height = 64, 64  # minimum acceptable dimensions
def is_image_large_enough(file_path):
    with Image.open(file_path) as img:
        return img.width >= min_width and img.height >= min_height
df = df[df['file'].apply(is_image_large_enough)]

# Image Preprocessing
# resizing and normalizing images
def preprocess_image(file_path):
    with Image.open(file_path) as img:
        img = img.resize((min_width, min_height))  # resize
        img = img.convert('RGB')  # ensure 3 channels
        img = np.array(img) / 255.0  # normalize to [0, 1]
    return img
df['image'] = df['file'].apply(preprocess_image)


NameError: name 'df' is not defined

In [8]:
print(df.columns) 
print(df.shape)  

NameError: name 'df' is not defined

In [9]:
#Face Detection
detector = dlib.get_frontal_face_detector()

def contains_face(file_path):
    img = cv2.imread(file_path)
    faces = detector(img, 1)
    return len(faces) > 0

df = df[df['file'].apply(contains_face)]

NameError: name 'dlib' is not defined

In [10]:
import urllib.request
import bz2

# URL of the file to be downloaded
url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"

# Path where the downloaded file will be stored
output_path = "shape_predictor_68_face_landmarks.dat.bz2"

# Download the file from `url` and save it locally under `output_path`:
urllib.request.urlretrieve(url, output_path)

# Open the .bz2 file for reading
with bz2.open(output_path, 'rb') as f:
    # Decompress the data
    decompressed_data = f.read()

# Write the decompressed data to a new file
with open('shape_predictor_68_face_landmarks.dat', 'wb') as f:
    f.write(decompressed_data)


In [11]:
# Face Alignment
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')  # Download this file

def align_face(file_path):
    img = cv2.imread(file_path)
    faces = detector(img, 1)
    for rect in faces:
        shape = predictor(img, rect)
        aligned_face = dlib.get_face_chip(img, shape)
        return aligned_face

df['aligned_face'] = df['file'].apply(align_face)

# Background Removal

# Lighting and Color Normalization
def normalize_image(face):
    gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    normalized = cv2.equalizeHist(gray)
    return normalized

df['normalized_face'] = df['aligned_face'].apply(normalize_image)

#Data Augmentation
def augment_image(face):
    M = cv2.getRotationMatrix2D((face.shape[1] / 2, face.shape[0] / 2), np.random.uniform(-30, 30), 1)
    rotated = cv2.warpAffine(face, M, (face.shape[1], face.shape[0]))
    return rotated

df['augmented_face'] = df['normalized_face'].apply(augment_image)

NameError: name 'dlib' is not defined

In [ ]:
Eencoding age
label_encoder = LabelEncoder()
df['ageLabel'] = label_encoder.fit_transform(df['age'])
df['ageLabel']

In [ ]:
def display_images(images, labels, num_images=5):
    fig, axes = plt.subplots(1, num_images, figsize=(15, 3))
    
    for i in range(num_images):
        axes[i].imshow(images[i])
        axes[i].set_title(f'Label: {labels[i]}')
        axes[i].axis('off')
    
    plt.show()

# Display the first 5 images from the training set
display_images(df['augmented_face'][:5], label_encoder.inverse_transform(df['ageLabel'])[:5])


# EDA and feature engineering

In [ ]:
#plotting the age and their disturbution


age_counts = df['label'].value_counts()

# Plotting
plt.figure(figsize=(12, 6))
age_counts.plot(kind='bar', color='skyblue')
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

NameError: name 'train_df' is not defined

In [ ]:
age_counts = df['age'].value_counts()
age_counts.index
age_counts.values
age_counts_df = pd.DataFrame({'age':age_counts.index,'Counts':age_counts.values})
px.bar(data_frame=age_counts_df,
 x='age',
 y='Counts',
 color='Counts',
 color_continuous_scale='blues',
 text_auto=True,
 title=f'Age Distribution')

In [ ]:
#showing images
import matplotlib.pyplot as plt

# Function to display images
def display_images(images, labels, num_images=5):
    fig, axes = plt.subplots(1, num_images, figsize=(15, 3))
    
    for i in range(num_images):
        axes[i].imshow(images[i])
        axes[i].set_title(f'Label: {labels[i]}')
        axes[i].axis('off')
    
    plt.show()

# Display the first 5 images from the training set
display_images(df['augmented_face'][:5], label_encoder.inverse_transform(df['ageLabel'])[:5])


NameError: name 'df' is not defined

In [ ]:
#Calculate mean and standard deviation along each channel

image_array = np.array(train_images)

mean_values = np.mean(image_array, axis=(0, 1, 2))
std_values = np.std(image_array, axis=(0, 1, 2))

# Create a DataFrame for better presentation
statistics_df = pd.DataFrame({
    'Channel': ['Red', 'Green', 'Blue'],
    'Mean': mean_values,
    'Standard Deviation': std_values
})

# Display the DataFrame with a styled format
styled_df = statistics_df.style.format({
    'Mean': '{:.2f}',
    'Standard Deviation': '{:.2f}'
})
styled_df

NameError: name 'np' is not defined

### skin texture

In [12]:
# Create a new column in the DataFrame to store texture features
df['texture_features'] = None

# Function to extract basic texture features from an image
def get_texture_features(image_path):
    # Load the image
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to smooth the image
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Compute texture features using the Laplacian operator
    laplacian = cv2.Laplacian(blurred, cv2.CV_64F)
    laplacian_var = np.var(laplacian)

    return laplacian_var

# Apply the get_texture_features function to each image in the DataFrame
texture_features_list = []
for image_path in tqdm(train_df['filepaths']):
    texture_features = get_texture_features(image_path)
    texture_features_list.append(texture_features)

# Add the extracted texture features to the DataFrame
df['texture_features'] = texture_features_list

# Visualize Texture Features Across Age Groups
age_groups = df.groupby('label')

# Set up subplots for visualization
fig, axs = plt.subplots(len(age_groups), figsize=(8, 5 * len(age_groups)))

for i, (age_group, group_data) in enumerate(age_groups):
    axs[i].set_title(f"Age Group: {age_group}")

    # Visualize texture features for a sample of images in the age group
    axs[i].hist(group_data['texture_features'], bins=20, color='skyblue', edgecolor='black')
    axs[i].set_xlabel('Texture Features')
    axs[i].set_ylabel('Frequency')

plt.show()


NameError: name 'df' is not defined

### wrinkle

In [13]:
# Create a new column in the DataFrame to store wrinkle features
df['wrinkle_features'] = None

# Function to extract basic wrinkle features from an image
def get_wrinkle_features(image_path):
    # Load the image
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Canny edge detection to enhance wrinkles
    edges = cv2.Canny(gray, 50, 150)

    # Compute the percentage of white pixels in the edges
    wrinkle_percentage = np.sum(edges) / (gray.shape[0] * gray.shape[1])

    return wrinkle_percentage

# Apply the get_wrinkle_features function to each image in the DataFrame
wrinkle_features_list = []
for image_path in tqdm(train_df['filepaths']):
    wrinkle_features = get_wrinkle_features(image_path)
    wrinkle_features_list.append(wrinkle_features)

# Add the extracted wrinkle features to the DataFrame
df['wrinkle_features'] = wrinkle_features_list

# Visualize Wrinkle Features Across Age Groups
age_groups = df.groupby('label')

# Set up subplots for visualization
fig, axs = plt.subplots(len(age_groups), figsize=(8, 5 * len(age_groups)))

for i, (age_group, group_data) in enumerate(age_groups):
    axs[i].set_title(f"Age Group: {age_group}")

    # Visualize wrinkle features for a sample of images in the age group
    axs[i].hist(group_data['wrinkle_features'], bins=20, color='lightcoral', edgecolor='black')
    axs[i].set_xlabel('Wrinkle Features')
    axs[i].set_ylabel('Frequency')

plt.show()

NameError: name 'df' is not defined

### hair color

In [14]:
#probably useless

def extract_hair_color(image_path):
    # Load the image
    image = cv2.imread(image_path)
    # Convert the image to HSV color space for better color analysis
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define a mask for the hair region in the HSV color space
    lower_hair_color = np.array([0, 10, 40])
    upper_hair_color = np.array([30, 200, 255])
    hair_mask = cv2.inRange(hsv_image, lower_hair_color, upper_hair_color)
    
    # Apply the hair mask to the original image
    hair_region = cv2.bitwise_and(image, image, mask=hair_mask)
    
    # Calculate the dominant hair color
    dominant_color = np.mean(hair_region, axis=(0, 1)).astype(int)
    
    return dominant_color

# Apply the extract_hair_color function to each image in the DataFrame
df['hair_color'] = df['filepaths'].apply(extract_hair_color)


NameError: name 'df' is not defined

## facial landmarks

In [15]:


# Load the pre-trained facial landmark predictor
predictor_path = "dlib/shape_predictor_68_face_landmarks.dat"  # Replace with the path to the shape predictor model
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

# Function to detect facial landmarks in an image
def detect_landmarks(image_pixels):
    # Convert the list of pixel values to a NumPy array
    image_array = np.array(image_pixels, dtype=np.uint8)

    # Detect faces in the image
    faces = detector(image_array)

    # Loop over each face and get facial landmarks
    landmarks_list = []
    for face in faces:
        shape = predictor(image_array, face)
        landmarks = [(shape.part(i).x, shape.part(i).y) for i in range(shape.num_parts)]
        landmarks_list.append(landmarks)

        # Draw landmarks on the image (comment out if you don't want to display)
        for (x, y) in landmarks:
            cv2.circle(image_array, (x, y), 2, (0, 255, 0), -1)

    # Display the image with landmarks (comment out if you don't want to display)
    cv2.imshow("Facial Landmarks", image_array)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return landmarks_list

# Apply the detect_landmarks function to each image in the DataFrame
df['landmarks'] = df['augmented_face'].apply(detect_landmarks)

# Now df['landmarks'] contains the detected facial landmarks for each image


NameError: name 'dlib' is not defined

In [16]:
#eye openness detection 

# Load the pre-trained facial landmark predictor
predictor_path = "dlib/shape_predictor_68_face_landmarks.dat"  # Replace with the path to the shape predictor model
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

# Function to detect facial landmarks and extract eye-related features
def detect_eye_features(image_pixels):
    # Convert the list of pixel values to a NumPy array
    image_array = np.array(image_pixels, dtype=np.uint8)

    # Ensure the image has 3 channels (for compatibility with cv2.COLOR_BGR2GRAY)
    if image_array.ndim == 2:
        image_array = cv2.cvtColor(image_array, cv2.COLOR_GRAY2BGR)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = detector(gray)

    # Check if a face is detected
    if len(faces) > 0:
        # Get facial landmarks for the first detected face
        shape = predictor(gray, faces[0])

        # Extract eye-related features
        left_eye_openness = shape.part(47).y - shape.part(43).y  # Example: vertical distance between eyebrow and lower eyelid
        right_eye_openness = shape.part(40).y - shape.part(38).y

       

    return left_eye_openness, right_eye_openness

    return None

# Example: Apply the detect_eye_features function to each image in the DataFrame
eye_features = df['normalized_face'].apply(detect_eye_features)

# Example: Add the extracted eye features to the DataFrame
df[['left_eye_openness', 'right_eye_openness']] = pd.DataFrame(eye_features.tolist(), index=df.index)


NameError: name 'dlib' is not defined

# Model

### cnn

In [ ]:
df= df.dropna()

In [18]:
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, mean_squared_error, mean_absolute_error
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input, Concatenate

# Assuming 'df' is your DataFrame

# Constants for image preprocessing
desired_width = 128
desired_height = 128
num_channels = 3  # Assuming color images, adjust if grayscale

# Step 1: Prepare Data
# Extract 'eye openness', 'texture features', and 'wrinkle features'
tabular_features = df_cleaned[['texture_features', 'wrinkle_features', 'left_eye_openness','right_eye_openness']]
# Standardize numerical features
scaler = StandardScaler()
tabular_features = scaler.fit_transform(tabular_features)

# Load and preprocess images
desired_channels = 3  # Assuming color images, adjust if grayscale

def preprocess_image(image):
    # Resize to a common size
    resized_image = cv2.resize(image, (desired_width, desired_height))
    # Ensure the image has the correct number of channels
    if resized_image.shape[-1] != desired_channels:
        # If the image has more than 3 channels, convert to grayscale
        resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
        # Add an extra dimension to represent the single channel
        resized_image = np.expand_dims(resized_image, axis=-1)
    # Normalize pixel values to be between 0 and 1
    resized_image = resized_image / 255.0
    # Add other preprocessing steps as needed (e.g., data augmentation)
    return resized_image

X_images = np.array([preprocess_image(image) for image in df_cleaned['image'].values])
# Extract target
y_target = df_cleaned['ageLabel'].values

# Split data into training and testing sets
X_images_train, X_images_test, X_tabular_train, X_tabular_test, y_train, y_test = train_test_split(
    X_images, tabular_features, y_target, test_size=0.2, random_state=42
)

# Step 2: Build Models
# Build CNN model for image feature extraction
image_input = Input(shape=(desired_width, desired_height, num_channels))
x = Conv2D(32, (3, 3), activation='relu')(image_input)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
image_output = Dense(128, activation='relu')(x)

# Build model for tabular features
tabular_input = Input(shape=(X_tabular_train.shape[1],))
tabular_output = Dense(128, activation='relu')(tabular_input)

# Step 3: Combine Models
concatenated = Concatenate(axis=-1)([image_output, tabular_output])
x = Dense(64, activation='relu')(concatenated)
output = Dense(1, activation='linear')(x)

combined_model = Model(inputs=[image_input, tabular_input], outputs=output)

# Step 4: Train and Evaluate
combined_model.compile(optimizer='adam', loss='mean_squared_error')
combined_model.fit([X_images_train, X_tabular_train], y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
y_pred = combined_model.predict([X_images_test, X_tabular_test])

# Calculate metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
# Assuming you have binary classification labels for f1_score
y_test_binary = (y_test > 0).astype(int)
y_pred_binary = (y_pred > 0).astype(int)
f1 = f1_score(y_test_binary, y_pred_binary)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'F1 Score: {f1}')


ModuleNotFoundError: No module named 'cv2'

Mean Absolute Error: 1.2200337251027424
Mean Squared Error: 2.036018026809009
F1 Score: 0.8679245283018869

### vggface

In [19]:
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, mean_squared_error, mean_absolute_error
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input, Concatenate
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input


# Constants for image preprocessing
desired_width = 128
desired_height = 128
num_channels = 3  # Assuming color images, adjust if grayscale

# Step 1: Prepare Data
# Extract 'eye openness', 'texture features', and 'wrinkle features'
tabular_features = df[['texture_features', 'wrinkle_features', 'left_eye_openness','right_eye_openness']]
# Standardize numerical features
scaler = StandardScaler()
tabular_features = scaler.fit_transform(tabular_features)

# Load and preprocess images
desired_channels = 3  # Assuming color images, adjust if grayscale

def preprocess_image(image):
    # Resize to a common size
    resized_image = cv2.resize(image, (desired_width, desired_height))
    # Ensure the image has the correct number of channels
    if resized_image.shape[-1] != desired_channels:
        # If the image has more than 3 channels, convert to grayscale
        resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
        # Add an extra dimension to represent the single channel
        resized_image = np.expand_dims(resized_image, axis=-1)
    # Normalize pixel values to be between 0 and 1
    resized_image = resized_image / 255.0
    # Add other preprocessing steps as needed (e.g., data augmentation)
    return resized_image

X_images = np.array([preprocess_image(image) for image in df_cleaned['image'].values])
# Extract target
y_target = df['ageLabel'].values

# Split data into training and testing sets
X_images_train, X_images_test, X_tabular_train, X_tabular_test, y_train, y_test = train_test_split(
    X_images, tabular_features, y_target, test_size=0.2, random_state=42
)

# Step 2: Build Models
# Build VGGFace model for image feature extraction
image_input = Input(shape=(desired_width, desired_height, num_channels))
vgg_model = VGGFace(model='vgg16', include_top=False, input_tensor=image_input)

# Freeze layers to retain pretrained weights
for layer in vgg_model.layers:
    layer.trainable = False

x = Flatten()(vgg_model.output)
image_output = Dense(128, activation='relu')(x)

# Build model for tabular features
tabular_input = Input(shape=(X_tabular_train.shape[1],))
tabular_output = Dense(128, activation='relu')(tabular_input)

# Step 3: Combine Models
concatenated = Concatenate(axis=-1)([image_output, tabular_output])
x = Dense(64, activation='relu')(concatenated)
output = Dense(1, activation='linear')(x)

combined_model = Model(inputs=[image_input, tabular_input], outputs=output)

# Step 4: Train and Evaluate
combined_model.compile(optimizer='adam', loss='mean_squared_error')
combined_model.fit([X_images_train, X_tabular_train], y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
y_pred = combined_model.predict([X_images_test, X_tabular_test])

# Calculate metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
# Assuming you have binary classification labels for f1_score
y_test_binary = (y_test > 0).astype(int)
y_pred_binary = (y_pred > 0).astype(int)
f1 = f1_score(y_test_binary, y_pred_binary)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'F1 Score: {f1}')


ModuleNotFoundError: No module named 'cv2'

Mean Absolute Error: 1.288718291123708
Mean Squared Error: 2.2314737071360065
F1 Score: 0.8679245283018869

# streamlit

In [20]:
#building up the model


def load_and_preprocess_data():
    # Load your data 
    zip_path = "dataset/age.zip"  
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall()
    df = pd.read_csv('age_detection.csv')
    image_path = df.loc[0, 'file']
    if not os.path.isabs(image_path):
        image_path = os.path.join(os.path.dirname(zip_path), image_path)
    df = df[df['file'].apply(lambda x: os.path.isfile(x))]
    # Duplicate Removal
    df.drop_duplicates(subset=['file'], keep='first', inplace=True)
    df = df[df['split'].isin(['train', 'test'])]    
    # Image Quality
    # removing images that are too small
    min_width, min_height = 64, 64  # minimum acceptable dimensions
    def is_image_large_enough(file_path):
        with Image.open(file_path) as img:
            return img.width >= min_width and img.height >= min_height
    df = df[df['file'].apply(is_image_large_enough)]

    # Image Preprocessing
    # resizing and normalizing images
    def preprocess_image(file_path):
        with Image.open(file_path) as img:
            img = img.resize((min_width, min_height))  # resize
            img = img.convert('RGB')  # ensure 3 channels
            img = np.array(img) / 255.0  # normalize to [0, 1]
        return img
    df['image'] = df['file'].apply(preprocess_image)
    detector = dlib.get_frontal_face_detector()

    def contains_face(file_path):
        img = cv2.imread(file_path)
        faces = detector(img, 1)
        return len(faces) > 0

    df = df[df['file'].apply(contains_face)]
    predictor = dlib.shape_predictor('dlib/shape_predictor_68_face_landmarks.dat')  # Download this file

    def align_face(file_path):
        img = cv2.imread(file_path)
        faces = detector(img, 1)
        for rect in faces:
            shape = predictor(img, rect)
            aligned_face = dlib.get_face_chip(img, shape)
            return aligned_face

    df['aligned_face'] = df['file'].apply(align_face)

    # Background Removal

    # Lighting and Color Normalization
    def normalize_image(face):
        gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        normalized = cv2.equalizeHist(gray)
        return normalized

    df['normalized_face'] = df['aligned_face'].apply(normalize_image)

    # Data Augmentation
    def augment_image(face):
        M = cv2.getRotationMatrix2D((face.shape[1] / 2, face.shape[0] / 2), np.random.uniform(-30, 30), 1)
        rotated = cv2.warpAffine(face, M, (face.shape[1], face.shape[0]))
        return rotated

    df['augmented_face'] = df['normalized_face'].apply(augment_image)
    label_encoder = LabelEncoder()
    df['ageLabel'] = label_encoder.fit_transform(df['age'])

    # Create a new column in the DataFrame to store texture features
    df['texture_features'] = None

    # Function to extract basic texture features from an image
    def get_texture_features(image_path):
        # Load the image
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to smooth the image
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # Compute texture features using the Laplacian operator
        laplacian = cv2.Laplacian(blurred, cv2.CV_64F)
        laplacian_var = np.var(laplacian)

        return laplacian_var

    # Apply the get_texture_features function to each image in the DataFrame
    texture_features_list = []
    for image_path in tqdm(df['file']):
        texture_features = get_texture_features(image_path)
        texture_features_list.append(texture_features)

    # Add the extracted texture features to the DataFrame
    df['texture_features'] = texture_features_list

    df['wrinkle_features'] = None

    # Function to extract basic wrinkle features from an image
    def get_wrinkle_features(image_path):
        # Load the image
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply Canny edge detection to enhance wrinkles
        edges = cv2.Canny(gray, 50, 150)

        # Compute the percentage of white pixels in the edges
        wrinkle_percentage = np.sum(edges) / (gray.shape[0] * gray.shape[1])

        return wrinkle_percentage

    # Apply the get_wrinkle_features function to each image in the DataFrame
    wrinkle_features_list = []
    for image_path in tqdm(df['file']):
        wrinkle_features = get_wrinkle_features(image_path)
        wrinkle_features_list.append(wrinkle_features)

    # Add the extracted wrinkle features to the DataFrame
    df['wrinkle_features'] = wrinkle_features_list

    # Load the pre-trained facial landmark predictor
    predictor_path = "dlib/shape_predictor_68_face_landmarks.dat"  # Replace with the path to the shape predictor model
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path)

    # Function to detect facial landmarks and extract eye-related features
    def detect_eye_features(image_pixels):
        # Convert the list of pixel values to a NumPy array
        image_array = np.array(image_pixels, dtype=np.uint8)

        # Ensure the image has 3 channels (for compatibility with cv2.COLOR_BGR2GRAY)
        if image_array.ndim == 2:
            image_array = cv2.cvtColor(image_array, cv2.COLOR_GRAY2BGR)

        # Convert the image to grayscale
        gray = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)

        # Detect faces in the image
        faces = detector(gray)

        # Check if a face is detected
        if len(faces) > 0:
            # Get facial landmarks for the first detected face
            shape = predictor(gray, faces[0])

            # Extract eye-related features
            left_eye_openness = shape.part(47).y - shape.part(43).y  # Example: vertical distance between eyebrow and lower eyelid
            right_eye_openness = shape.part(40).y - shape.part(38).y

            return left_eye_openness, right_eye_openness

    # Example: Apply the detect_eye_features function to each image in the DataFrame
    eye_features = df['normalized_face'].apply(detect_eye_features)

    # Example: Add the extracted eye features to the DataFrame
    df[['left_eye_openness', 'right_eye_openness']] = pd.DataFrame(eye_features.tolist(), index=df.index)
    df = df.dropna()

    return df

def load_model(df_cleaned):
    # Load the trained model
    desired_width = 128
    desired_height = 128
    num_channels = 3  # Assuming color images, adjust if grayscale

    # Step 1: Prepare Data
    # Extract 'eye openness', 'texture features', and 'wrinkle features'
    tabular_features = df_cleaned[['texture_features', 'wrinkle_features', 'left_eye_openness','right_eye_openness']]
    # Standardize numerical features
    scaler = StandardScaler()
    tabular_features = scaler.fit_transform(tabular_features)

    # Load and preprocess images
    desired_channels = 3  # Assuming color images, adjust if grayscale

    def preprocess_image(image):
        # Resize to a common size
        resized_image = cv2.resize(image, (desired_width, desired_height))
        # Ensure the image has the correct number of channels
        if resized_image.shape[-1] != desired_channels:
            # If the image has more than 3 channels, convert to grayscale
            resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
            # Add an extra dimension to represent the single channel
            resized_image = np.expand_dims(resized_image, axis=-1)
        # Normalize pixel values to be between 0 and 1
        resized_image = resized_image / 255.0
        # Add other preprocessing steps as needed (e.g., data augmentation)
        return resized_image

    X_images = np.array([preprocess_image(image) for image in df_cleaned['image'].values])
    # Extract target
    y_target = df_cleaned['ageLabel'].values

    # Split data into training and testing sets
    X_images_train, X_images_test, X_tabular_train, X_tabular_test, y_train, y_test = train_test_split(
        X_images, tabular_features, y_target, test_size=0.2, random_state=42
    )

    # Step 2: Build Models
    # Build CNN model for image feature extraction
    image_input = Input(shape=(desired_width, desired_height, num_channels))
    x = Conv2D(32, (3, 3), activation='relu')(image_input)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    image_output = Dense(128, activation='relu')(x)

    # Build model for tabular features
    tabular_input = Input(shape=(X_tabular_train.shape[1],))
    tabular_output = Dense(128, activation='relu')(tabular_input)

    # Step 3: Combine Models
    concatenated = Concatenate(axis=-1)([image_output, tabular_output])
    x = Dense(64, activation='relu')(concatenated)
    output = Dense(1, activation='linear')(x)

    combined_model = Model(inputs=[image_input, tabular_input], outputs=output)

    # Step 4: Train and Evaluate
    combined_model.compile(optimizer='adam', loss='mean_squared_error')
    combined_model.fit([X_images_train, X_tabular_train], y_train, epochs=10, batch_size=32, validation_split=0.2)

    # Evaluate the model
    y_pred = combined_model.predict([X_images_test, X_tabular_test])

    return combined_model

def preprocess_input(user_input):
    # Assuming user_input is a preprocessed image (e.g., aligned and normalized)
    processed_input = user_input.resize((desired_width, desired_height))  # Resize if needed
    processed_input = np.array(processed_input) / 255.0  # Normalize to [0, 1]
    processed_input = np.expand_dims(processed_input, axis=0)  # Add batch dimension
    return processed_input

def make_prediction(model, processed_input):
    # Assuming processed_input is a tuple containing (image_input, tabular_input)
    image_input, tabular_input = processed_input

    # Make predictions using the model
    prediction = model.predict([image_input, tabular_input])

    return prediction

ModuleNotFoundError: No module named 'cv2'

In [21]:
# Save the trained model
age_predicton = load_model(load_and_preprocess_data())
age_predicton.save('saved_model.h5')

NameError: name 'load_and_preprocess_data' is not defined

In [22]:
# model.py

from keras.models import load_model

def load_age_model():
    
    # Load the trained model
    model = load_model('saved_model.h5')  # Adjust the filename based on your saved model
    return model

In [23]:
# streamlit_app.py

import streamlit as st
from PIL import Image
import numpy as np

# Load the model
model = load_age_model()

def main():
    st.title("Age Detection App")

    # File uploader for image input
    uploaded_file = st.file_uploader("Choose an image...", type="jpg")

    if uploaded_file is not None:
        # Display the uploaded image
        image = Image.open(uploaded_file)
        st.image(image, caption="Uploaded Image", use_column_width=True)

        # Resize the image to match the model's expected input size
        resized_image = image.resize((128, 128))  # Adjust dimensions based on your model
        st.image(resized_image, caption="Resized Image", use_column_width=True)

        # Preprocess the input
        processed_input = preprocess_input(resized_image)

        # Make a prediction
        prediction = make_prediction(model, processed_input)

        # Display the result
        # Display the result
        st.write(f"Predicted Age: {model.prediction[0]}")


if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'streamlit'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0d3193c3-f84c-4987-b2bb-da138af83f70' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>